# Initial Data Processing

In [ ]:
#Downloaded the data from the GLODAP website
taa = pd.read_csv("~/hollings/data/GLODAPv2.2022_Merged_Master_File.csv", low_memory = False)

#Selecting relevant columns
taa_ed = taa[['G2cruise', 'G2region', 'G2latitude', 'G2longitude', 'G2bottomdepth', 'G2depth', 'G2temperature', 'G2salinity', 'G2oxygen', 'G2nitrate', 'G2silicate', 'G2aou', 'G2theta', 'G2talk', "G2year", "G2month", "G2day"]] 

### Added and editing variables

In [ ]:
#Initial Data Processing
ta_initial = pd.read_csv("~/hollings/data/TA_PreProcessed_PacificOnly2.csv")
ta_bd = pd.read_csv("~/hollings/data/Output_NOGOMMED_060524.csv")

#Adding bottom depth 
cr = ta_initial["Cruise"].unique()
lats = ta_initial["Latitude"].unique()
lons = ta_initial["Longitude"].unique()
taa = ta_initial["TA"].unique()

bd = ta_bd[ta_bd.Cruise.isin(cr) & ta_bd.Latitude.isin(lats) & ta_bd.Longitude.isin(lons) & ta_bd.TA.isin(taa)].reset_index()

ta_initial["Bottom Depth"] = bd["Bottom Depth"]

#Adding row number col
ta_initial = ta_initial.assign(order=range(len(ta_initial)))

#Adding AOU and PT using sw lib for continuity with ESPERs
latitude = list(ta_initial['Latitude'])
depth = list(ta_initial['Depth'])
salinity = list(ta_initial['S'])
temperature = list(ta_initial['T'])
oxygen = list(ta_initial['Oxygen'])
ptemp= gsw.ptmp(salinity, temperature, gsw.pres(depth, latitude), pr=0)
aou = gsw.satO2(salinity, ptemp)*44.6596 - (oxygen)
ta_initial["PT"] = ptemp
ta_initial["AOU"] = aou

#Additional columns
ta_sorteddepth=ta_initial.copy()
ta_sorteddepth = ta_sorteddepth.sort_values(by= "Depth")

#Addition of SST column 
ta_sorteddepth["SST"] = ta_sorteddepth.groupby(["Cruise", "Latitude", "Longitude", "year", "month", "day"])["T"].transform("first")                                                               

#Addition of sea surface salinity col
ta_sorteddepth["SSS"] = ta_sorteddepth.groupby(["Cruise", "Latitude", "Longitude", "year", "month", "day"])["S"].transform("first")

ta_ed = ta_sorteddepth.sort_index()